## Surviving and recovering from failures 

It's time to deploy and manage our model! Now that your client is happy with your work, we'll pick up from BLU13 and provide them with a small app to use it.

In [1]:
import os
import pandas as pd
import json
import joblib
import pickle
import requests
from uuid import uuid4
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score
from category_encoders import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

/Users/catarinasilva/.pyenv/versions/3.8.5/envs/batch4-blu14/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Time to deploy! 

If you're reacting more or less like this 

<img src="media/model-deploy-unknown.png" width=300 />

do not panic, if you still haven't really internalized the last BLU, we provide you with the template code to deploy the model that we created in part 1, which you can also reuse as a starting point for the exercises. This code handles:

* deserialization of our model
* serving predictions 
* storage of observations
* update of observations

In the previous BLU you've learned how to deploy in heroku, and you will want to do that to server your app. However, for the following topics we will focus on testing locally.

<br>

### Deploying locally

What does this mean?

Well, it means we'll launch a server in our own machine, making it available to test it there, but not available to the world. This server will be accessible to you by the URLs `127.0.0.1` or `localhost`. These are reserved so that in every machine the traffic that you send to these is looped back.


<img src="media/localhost-ben.png" width=450 />





So start by running the server we provide under `server.py`. Open a shell tab and go the BLU folder. Once you're there, run the following to start up the template server

```sh

python server.py


```

You should see something like this if everything went well:

<img src="media/flask-server-log.png" width="100%" />

Yes, even with that scary red warning this is fine, you're ready to continue. The next thing we will do is to send some requests to our server. 


### Sending some observations

If you remember correctly we used the following columns:

* SubjectRaceCode
* SubjectSexCode
* SubjectEthnicityCode
* StatuteReason
* InterventionReasonCode
* ResidentIndicator
* SearchAuthorizationCode
* SubjectAge
* hour
* day_of_week

And the way we need to communicate to our server is by sending a json object such as:

```json

{
  "id": "your-observation-id",
  "observation": {
     "SubjectRaceCode": "W",
     "SubjectSexCode": "F",
     "SubjectEthnicityCode": "H",
     "StatuteReason": "Stop sign", 
     "InterventionReasonCode": "V", 
     "ResidentIndicator": False, 
     "SearchAuthorizationCode": "N",
     "SubjectAge": 20,
     "hour": 20,
     "day_of_week": "Tuesday",
   }
}

```

We'll do this in 2 ways:

* by using cURL requests, which you used in the previous BLU
* by using the `requests` library from python

We'll start by creating our dummy observation as JSON and print so we can use it in curl:

In [2]:
observation = {
  "id": "fake-observation-{}".format(uuid4()),
  "observation": {
      "SubjectRaceCode": "B",
      "SubjectSexCode": "M",
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }
}

print(json.dumps(observation))

{"id": "fake-observation-cc2cd636-18fe-4dc3-8681-b310ea37390d", "observation": {"SubjectRaceCode": "B", "SubjectSexCode": "M", "SubjectEthnicityCode": "N", "StatuteReason": "Stop sign", "InterventionReasonCode": "V", "ResidentIndicator": false, "SearchAuthorizationCode": "N", "SubjectAge": 20, "hour": 20, "day_of_week": "Tuesday"}}


Let's copy this and prepare our curl request:


```sh

curl -X POST http://localhost:5000/predict -d '{"id": "fake-observation-1bc5145b-d7b5-2688-95e8-21a378204133", "observation": {"SubjectRaceCode": "B", "SubjectSexCode": "M", "SubjectEthnicityCode": "N", "StatuteReason": "Stop sign", "InterventionReasonCode": "V", "ResidentIndicator": false, "SearchAuthorizationCode": "N", "SubjectAge": 20, "hour": 20, "day_of_week": "Tuesday"}}'  -H "Content-Type:application/json"


```

**Note**: everytime you run the previous cell, a different ID will be generated. You can also chose to change your id if you want to perform the request again.

If you sent the request through cURL, re-run the observation cell and let's run the same request through the requests library. This should give you the exact same probability and prediction as the curl request

In [3]:
observation = {
  "id": "fake-observation-{}".format(uuid4()),
  "observation": {
      "SubjectRaceCode": "B",
      "SubjectSexCode": "M",
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }
}


url="http://127.0.0.1:5000/predict"
headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(observation), headers=headers)

print(r.status_code)
print(r.text)

200
{"prediction":true,"proba":0.5323205446823367}



Pretty simple, right? Try out a few more requests and play around with both commands

**Note**: You should get a status 200 and a proper response here. If not, make sure you ran the server as mentioned before and that you did so with the environment activated.

<br>

## Dealing with unexpected formats


But what happens if we get a weird observation? 

Let's start by changing our dictionary so that the `observation` is now `my_observation`



In [4]:
# Bad format

observation = {
  "id": "fake-observation-{}".format(uuid4()),
  "my_observation": {
      "SubjectRaceCode": "B",
      "SubjectSexCode": None,
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }
}


url="http://127.0.0.1:5000/predict"
headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(observation), headers=headers)

print(r.status_code)
print(r.text)

500
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>



<img src="media/michael-surprised.jpg" width=500 />


Ok ok, your users know what they should send.

But what about if they miss out some columns?


In [5]:
# Missing columns

observation = {
  "id": "fake-observation-{}".format(uuid4()),
  "observation": {
      "SubjectAge": 20,
      "hour": 20
  }
}


url="http://127.0.0.1:5000/predict"
headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(observation), headers=headers)

print(r.status_code)
print(r.text)

200
{"prediction":true,"proba":0.5021033711778715}




<img src="media/michael-nope.jpg" width=500 />

We got a 200 but in reality our model had almost no information about the observation, how do we know how to interpret this information? 

Maybe it is useful to see what happens if we send just complete nonsense? 

In [6]:
# Non sense values that don't break the request

observation = {
  "id": "fake-observation-{}".format(uuid4()),
  "observation": {
      "SubjectRaceCode": "A",
      "SubjectSexCode": "B",
      "SubjectEthnicityCode": "C",
      "StatuteReason": "D", 
      "InterventionReasonCode": "E", 
      "ResidentIndicator": "F", 
      "SearchAuthorizationCode": "F",
      "SubjectAge": 1,
      "hour": 90,
      "day_of_week": "potato",
  }
}


url="http://127.0.0.1:5000/predict"
headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(observation), headers=headers)

print(r.status_code)
print(r.text)

200
{"prediction":true,"proba":0.5036075609758188}




<img src="media/michael-cringe.png" width=500 />

Getting a 200 is not always good. In fact, if someone just sends completely random values and get a probability and prediction, not only are they mislead, but since our system is also storing these observations, we will get poluted data. 

**Most of the times, silent errors are worse than explicit ones** 

Let's do one more just so you get the full picture:

In [7]:
# More non sense values that break the request

observation = {
  "id": "fake-observation-{}".format(uuid4()),
  "observation": {
      "SubjectRaceCode": "B",
      "SubjectSexCode": "M",
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SubjectAge": "twenty",
      "hour": 20,
      "day_of_week": "Tuesday",
  }
}


url="http://127.0.0.1:5000/predict"
headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(observation), headers=headers)

print(r.status_code)
print(r.text)

500
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>



<img src="media/michael-tired.jpg" width=500 />

Alright, alright, I'll stop. By this point, you should have an idea of the different ways input data can be bad. So what do we do?


### Handling input 


We protect the code. On one hand, we obviously don't want to send obscure errors to our client. On the other hand, we don't want to store data that doesn't make sense. So neither of the errors above are acceptable.

Let's retrieve the map we did with our known values from the last unit:


In [8]:
known_categories = {
    "InterventionReasonCode": {"values": ["V", "E", "I"], "default": None},
    "SubjectRaceCode": {"values": ["W", "B", "A", "I"], "default": None},
    "SubjectSexCode": {"values": ["M", "F"], "default": None},
    "SubjectEthnicityCode": {"values": ["H", "M", "N"], "default": "N"},
    "SearchAuthorizationCode": {"values": ["O", "I", "C", "N"], "default": "N"},
    # We can use it also for booleans!
    "TownResidentIndicator": {"values": [True, False]}, 
    "ResidentIndicator": {"values": [True, False]},
    "VehicleSearchedIndicator": {"values": [True, False]},
    "ContrabandIndicator": {"values": [True, False]},
}


We're going to do some slight modifications. Since we ended up using only a subset of our features and we actually augmented it with some others, we'll change the map to reflect this. Additionally, we'll keep only the allowed values and forget about any defaults for now.

Notice that we are not handling numeric values, only categorical (including boolean) values in this map. We'll go back to the numerical values later on


In [9]:
valid_categories = {
    "InterventionReasonCode": ["V", "E", "I"],
    "SubjectRaceCode": ["W", "B", "A", "I"],
    "SubjectSexCode": ["M", "F"],
    "SubjectEthnicityCode": ["H", "M", "N"],
    "SearchAuthorizationCode": ["O", "I", "C", "N"],
    "ResidentIndicator": {"values": [True, False]},
    "StatuteReason": [
        'Stop Sign', 'Other', 'Speed Related', 'Cell Phone', 'Traffic Control Signal', 'Defective Lights', 
        'Moving Violation', 'Registration', 'Display of Plates', 'Equipment Violation', 'Window Tint', 
        'Suspended License', 'Seatbelt', 'Other/Error', 'STC Violation', 'Administrative Offense', 'Unlicensed Operation'], 
    "day_of_week": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
}


What we want to do is verify, when we receive the input, that it is:

1. a valid input, with an `id` and an `observation`
2. valid columns under the observation
3. if categorical, a valid category within its column

We can build some small functions to do that for us

In [10]:
def check_request(request):
    """
        Validates that our request is well formatted
        
        Returns:
        - assertion value: True if request is ok, False otherwise
        - error message: empty if request is ok, False otherwise
    """
    
    if "id" not in request:
        error = "Field `id` missing from request: {}".format(request)
        return False, error
    
    if "observation" not in request:
        error = "Field `observation` missing from request: {}".format(request)
        return False, error
    
    return True, ""



def check_valid_column(observation):
    """
        Validates that our observation only has valid columns
        
        Returns:
        - assertion value: True if all provided columns are valid, False otherwise
        - error message: empty if all provided columns are valid, False otherwise
    """
    
    valid_columns = {
      "SubjectRaceCode",
      "SubjectSexCode",
      "SubjectEthnicityCode",
      "StatuteReason", 
      "InterventionReasonCode", 
      "ResidentIndicator", 
      "SearchAuthorizationCode",
      "SubjectAge",
      "hour",
      "day_of_week",
    }
    
    keys = set(observation.keys())
    
    if len(valid_columns - keys) > 0: 
        missing = valid_columns - keys
        error = "Missing columns: {}".format(missing)
        return False, error
    
    if len(keys - valid_columns) > 0: 
        extra = keys - valid_columns
        error = "Unrecognized columns provided: {}".format(extra)
        return False, error    

    return True, ""



def check_categorical_values(observation):
    """
        Validates that all categorical fields are in the observation and values are valid
        
        Returns:
        - assertion value: True if all provided categorical columns contain valid values, 
                           False otherwise
        - error message: empty if all provided columns are valid, False otherwise
    """
    
    valid_category_map = {
        "InterventionReasonCode": ["V", "E", "I"],
        "SubjectRaceCode": ["W", "B", "A", "I"],
        "SubjectSexCode": ["M", "F"],
        "SubjectEthnicityCode": ["H", "M", "N"],
        "SearchAuthorizationCode": ["O", "I", "C", "N"],
        "ResidentIndicator": [True, False],
        "StatuteReason": [
            'Stop Sign', 'Other', 'Speed Related', 'Cell Phone', 'Traffic Control Signal', 'Defective Lights', 
            'Moving Violation', 'Registration', 'Display of Plates', 'Equipment Violation', 'Window Tint', 
            'Suspended License', 'Seatbelt', 'Other/Error', 'STC Violation', 'Administrative Offense', 'Unlicensed Operation'], 
        "day_of_week": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    }
    
    for key, valid_categories in valid_category_map.items():
        if key in observation:
            value = observation[key]
            if value not in valid_categories:
                error = "Invalid value provided for {}: {}. Allowed values are: {}".format(
                    key, value, ",".join(["'{}'".format(v) for v in valid_categories]))
                return False, error
        else:
            error = "Categorical field {} missing"
            return False, error

    return True, ""


Let's try out our functions:

#### Check request structure


In [11]:
check_request({"id": "fake-id", "observation": "fake-obs"})


(True, '')

In [12]:
check_request({"bad_id": "fake-id", "observation": "fake-obs"})


(False,
 "Field `id` missing from request: {'bad_id': 'fake-id', 'observation': 'fake-obs'}")

#### Check observation


In [13]:
observation = {
      "SubjectRaceCode": "B",
      "SubjectSexCode": "M",
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }

check_valid_column(observation)

(True, '')

In [14]:
observation = {
      "SubjectRaceCode": "B",
      "SubjectSexCode": "M",
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }

check_valid_column(observation)

(False, "Missing columns: {'SearchAuthorizationCode'}")

In [15]:
observation = {
      "SubjectRaceCode": "B",
      "SubjectSexCode": "M",
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SomeRandomColumn": "N",
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }

check_valid_column(observation)

(False, "Unrecognized columns provided: {'SomeRandomColumn'}")

#### Check categories


In [16]:
observation = {
      "SubjectRaceCode": "B",
      "SubjectSexCode": "M",
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop Sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SomeRandomColumn": "N",
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }

check_categorical_values(observation)

(True, '')

In [17]:
observation = {
      "SubjectRaceCode": "t",
      "SubjectSexCode": "M",
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SomeRandomColumn": "N",
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }

check_categorical_values(observation)

(False,
 "Invalid value provided for SubjectRaceCode: t. Allowed values are: 'W','B','A','I'")

In [18]:
observation = {
      "SubjectRaceCode": "B",
      "SubjectSexCode": "M",
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": "COUCH POTATO", 
      "SearchAuthorizationCode": "N",
      "SomeRandomColumn": "N",
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }

check_categorical_values(observation)

(False,
 "Invalid value provided for ResidentIndicator: COUCH POTATO. Allowed values are: 'True','False'")

<br>

#### Numerical values

What about our numeric values - `hour` and `SubjectAge`. Well, each has a particular set of conditions that make sense to apply, so let's create some functions to perform similar verifications:



In [19]:
def check_hour(observation):
    """
        Validates that observation contains valid hour value 
        
        Returns:
        - assertion value: True if hour is valid, False otherwise
        - error message: empty if hour is valid, False otherwise
    """
    
    hour = observation.get("hour")
        
    if not hour:
        error = "Field `hour` missing"
        return False, error

    if not isinstance(hour, int):
        error = "Field `hour` is not an integer"
        return False, error
    
    if hour < 0 or hour > 24:
        error = "Field `hour` is not between 0 and 24"
        return False, error

    return True, ""


def check_age(observation):
    """
        Validates that observation contains valid hour value 
        
        Returns:
        - assertion value: True if hour is valid, False otherwise
        - error message: empty if hour is valid, False otherwise
    """
    
    age = observation.get("SubjectAge")
        
    if not age: 
        error = "Field `SubjectAge` missing"
        return False, error

    if not isinstance(age, int):
        error = "Field `SubjectAge` is not an integer"
        return False, error
    
    if age < 10 or age > 100:
        error = "Field `SubjectAge` is not between 10 and 100"
        return False, error

    return True, ""


In [20]:
observation = {
      "hour": 20,
      "day_of_week": "Tuesday",
  }

check_hour(observation)

(True, '')

In [21]:
observation = {
      "hour": 100,
      "day_of_week": "Tuesday",
  }

check_hour(observation)

(False, 'Field `hour` is not between 0 and 24')

In [22]:
observation = {
      "SubjectAge": 20,
      "day_of_week": "Tuesday",
  }

check_age(observation)

(True, '')

### Putting it all together

Now we can run our server with these functions. 

Run the code under `protected_server.py`

```sh

python protected_server.py


```


And try out the same examples as before to see what the server returns to us:


In [24]:
# Bad format

observation = {
  "id": "fake-observation-{}".format(uuid4()),
  "my_observation": {
      "SubjectRaceCode": "B",
      "SubjectSexCode": None,
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }
}


url="http://127.0.0.1:5000/predict"
headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(observation), headers=headers)

print(r.text)

{"error":"Field `observation` missing from request: {'id': 'fake-observation-36de12eb-4b3d-43fa-9200-9edee8e1e44b', 'my_observation': {'SubjectRaceCode': 'B', 'SubjectSexCode': None, 'SubjectEthnicityCode': 'N', 'StatuteReason': 'Stop sign', 'InterventionReasonCode': 'V', 'ResidentIndicator': False, 'SearchAuthorizationCode': 'N', 'SubjectAge': 20, 'hour': 20, 'day_of_week': 'Tuesday'}}"}



In [25]:
# Missing columns

observation = {
  "id": "fake-observation-{}".format(uuid4()),
  "observation": {
      "SubjectRaceCode": "B",
      "StatuteReason": "Stop sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SubjectAge": 20,
      "hour": 20,
      "day_of_week": "Tuesday",
  }
}


url="http://127.0.0.1:5000/predict"
headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(observation), headers=headers)

print(r.text)


{"error":"Missing columns: {'SubjectEthnicityCode', 'SubjectSexCode'}"}



In [26]:
# Non sense values that don't break the request

observation = {
  "id": "fake-observation-{}".format(uuid4()),
  "observation": {
      "SubjectRaceCode": "A",
      "SubjectSexCode": "B",
      "SubjectEthnicityCode": "C",
      "StatuteReason": "D", 
      "InterventionReasonCode": "E", 
      "ResidentIndicator": "F", 
      "SearchAuthorizationCode": "F",
      "SubjectAge": 1,
      "hour": 90,
      "day_of_week": "potato",
  }
}


url="http://127.0.0.1:5000/predict"
headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(observation), headers=headers)

print(r.status_code)
print(r.text)

200
{"error":"Invalid value provided for SubjectSexCode: B. Allowed values are: 'M','F'"}



In [27]:
# Non sense values that break the request

observation = {
  "id": "fake-observation-{}".format(uuid4()),
  "observation": {
      "SubjectRaceCode": "B",
      "SubjectSexCode": "M",
      "SubjectEthnicityCode": "N",
      "StatuteReason": "Stop Sign", 
      "InterventionReasonCode": "V", 
      "ResidentIndicator": False, 
      "SearchAuthorizationCode": "N",
      "SubjectAge": "twenty",
      "hour": 20,
      "day_of_week": "Tuesday",
  }
}


url="http://127.0.0.1:5000/predict"
headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(observation), headers=headers)

print(r.status_code)
print(r.text)

200
{"error":"Field `SubjectAge` is not an integer"}




<img src="media/great_success.jpg" width=400 />


Now we have a bit more robust server. Try out other examples to see if you can break the server!


#### A final note

When designing APIs there are actually proper error codes to apply for each type of error. For example here the correct code might be 422 (`Unprocessable Entity`) while for the previously coded error when the id is the same we may want to return a 409 (`Conflict`). However, you don't need to know these for now

<a href="https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/418"><img src="media/418_teapot.jpeg" width=400 /></a>

_Who said programmers are no fun?_


<br>

## Monitoring and maintaining the system

So you've deployed your model, you made sure to protect it from bad input, you even found a bunch of other potential sources of issues and protected them. 

What next?


### Uptime and surviving failures


Even when you are 100% sure all you did was right, things can go wrong. After all, just because your model is deployed on the "cloud" it doesn't mean it doesn't use resources.

You can have all sorts of problems related to:

* CPU usage
* Memory usage
* Database connections

And the list could go on? So how does the average developer handle all of this?




### Testing

Another important part for you to understand is that even when you are 100% sure all you did was right, you probably did something wrong. 

> Anything that can go wrong, will go wrong

(this is called Murphy's law)


So usually we perform some sort of testing on our server and model before we actually share it with the client. This usually happens before you deploy the model to what is called a production environment - this is, the same environment where your app is available to the client.

<img src="media/testing-in-production.png" width=400 />

There are several layers of testing that you can apply, but we'll just show a couple of examples to give you an overview of how testing can help you find unwanted behaviors. After all, you don't want your server to only stay alive, you want it to follow a certain behavior that you defined. 



In [ ]:
# Show small examples of unit tests -> example some of the check functions

Final remarks on testing

### Performance requirements

* query per second


suggestions for improving:

* caching
